# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_df = pd.read_csv("../weather_data_df.csv", encoding="utf-8")
del weather_df["Unnamed: 0"]
weather_df.head()

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,georgetown,5.41,100.34,86.00,66,20,5.82,MY,1596031223
1,samarai,-10.62,150.67,77.14,85,99,17.25,PG,1596031513
2,ankang,32.68,109.02,75.00,51,100,1.01,CN,1596031515
3,hambantota,6.12,81.12,84.20,74,20,8.05,LK,1596031518
4,nanortalik,60.14,-45.24,46.72,93,100,9.91,GL,1596031520


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [12]:
fig = gmaps.figure(center=(30.0, -10.0), zoom_level=2)
heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=2)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
ideal_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] <80) & (weather_df["Wind Speed"] < 3), :]
ideal_df = ideal_df.dropna(how="any")

ideal_df.reset_index(inplace=True)
del ideal_df["index"]

ideal_df

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,ankang,32.68,109.02,75.00,51,100,1.01,CN,1596031515
1,albany,42.60,-73.97,77.00,64,0,2.91,US,1596031518
2,sao filipe,14.90,-24.50,78.64,82,100,2.66,CV,1596031621
3,ponta do sol,32.67,-17.10,73.00,62,21,2.66,PT,1596031775
4,lashio,22.93,97.75,70.05,96,100,2.04,MM,1596032024
5,mrakovo,52.72,56.62,71.55,77,100,1.30,RU,1596032125
6,saint george,37.10,-113.58,75.00,16,1,1.90,US,1596032213
7,barkot,30.82,78.20,70.68,81,100,2.21,IN,1596032258
8,wajima,37.40,136.90,73.58,88,95,2.66,JP,1596032280
9,pathein,16.78,94.73,78.33,91,100,1.39,MM,1596032351


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
#Create hotel list
hotel_list = []

#Create loop for hotel list
for city in range(len(ideal_df)):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    jsn = response.json()
    try:
        hotel_list.append(jsn["results"][0]["name"])
    except:
        hotel_list.append("")

ideal_df["Hotel Name"] = hotel_list

ideal_df = ideal_df.dropna(how="any")
ideal_df.head()

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,ankang,32.68,109.02,75.00,51,100,1.01,CN,1596031515,Ankang
1,albany,42.60,-73.97,77.00,64,0,2.91,US,1596031518,Voorheesville
2,sao filipe,14.90,-24.50,78.64,82,100,2.66,CV,1596031621,Sao Filipe
3,ponta do sol,32.67,-17.10,73.00,62,21,2.66,PT,1596031775,Ribeira Brava
4,lashio,22.93,97.75,70.05,96,100,2.04,MM,1596032024,Lashio


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))